<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/2_GPT_multiclass_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter

## Clasificacion multietiquetas con GPT 4 mini (sin finetunning) por batch


**Natalia Dedandi**





La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction



In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [2]:
!pip install datasets seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 require

In [3]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [4]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

Debo cargar mi dataset de TEST y pasarle un archivo BATCH para que lo procese

Usare este mensaje para cada Tweet:

"role": "system", "content": "You are trained to analyze and detect the sentiment of the given text. Only hateful or not hateful is allowed. The answer should be in spanish"

In [5]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/36420 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11343 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/9106 [00:00<?, ? examples/s]

In [6]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 36420
    })
    test: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 11343
    })
    dev: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 9106
    })
})

In [7]:
#tomo una muestra de 100 TW random
df_test = pd.DataFrame(ds1['test'])
df_test

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
0,397703,Mora Godoy cierra su escuela de tango y remata...,@usuario Ojalá se funda y cague de hambre,Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
1,397704,Mora Godoy cierra su escuela de tango y remata...,@usuario Faltó que se enganchó al hijo de sofo...,Mora Godoy cierra su escuela de tango y remata...,1,Puede marcar con justeza la fecha de inicio de...,0,1,0,0,0,0,0,0,0
2,397705,Mora Godoy cierra su escuela de tango y remata...,"@usuario y bue, que le vaya a llorar a Cristin...",Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
3,397706,Mora Godoy cierra su escuela de tango y remata...,@usuario @usuario Jodete. Macri te hizo conoce...,Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
4,397707,Mora Godoy cierra su escuela de tango y remata...,"@usuario Que se joda, es su gobierno el que el...",Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11338,361071,Nahir Galarza sumó a una abogada feminista a s...,@usuario Con esa varita q no rompe un plato ni...,Nahir Galarza sumó a una abogada feminista a s...,0,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0
11339,361072,Nahir Galarza sumó a una abogada feminista a s...,@usuario No las entiendo a estas pelotudas. So...,Nahir Galarza sumó a una abogada feminista a s...,1,"La defensa de Nahir Galarza, condenada el 3 de...",0,1,0,0,0,0,0,0,0
11340,361073,Nahir Galarza sumó a una abogada feminista a s...,@usuario Lo mató por qué es macho!,Nahir Galarza sumó a una abogada feminista a s...,0,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0
11341,361074,Nahir Galarza sumó a una abogada feminista a s...,@usuario Cuando sos asesina tenés que hacerte ...,Nahir Galarza sumó a una abogada feminista a s...,0,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0


Documentacion para el trabajo en batch:
https://platform.openai.com/docs/guides/batch/getting-started


Se prepara el formato completo del archivo para batch con el siguiente formato:

{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}


Al rol System se le envía la siguiente consigna:

You must classify a text with one or more of the following labels: Label WOMEN discrimination against women, label LGBTI gender discrimination, specifically against gays, lesbians or transsexuals,label RACISM is racism, or discrimination against people according to their skin color,label
CLASS is discrimination for being poor or living in a marginal or poor place, label POLITICS is hatred of politicians, label DISABLED is discrimination against people with disabilities, label APPEARENCE is discrimination against people because of their physical appearance, specifically fatphobia, label CRIMINAL is discrimination against people because of a criminal record or links to criminality and label CALLS is hateful text that incites violence
If the text is not hatefull indicate label NOT HATEFUL

In [20]:
# Specify the columns
id_column = 'id'
text_column = 'text'

# Convert the dataframe into a list of dictionaries with custom_id and messages
batch = [
    {
        'custom_id': str(row[id_column]),
        'method': 'POST',
        'url': '/v1/chat/completions',
        'body':{
            'model': 'gpt-4o-mini',
            'messages': [
                {'role': 'system', 'content': "You must classify a text with one or more of the following labels: Label WOMEN discrimination against women, label LGBTI gender discrimination, specifically against gays, lesbians or transsexuals,label RACISM is racism, or discrimination against people according to their skin color,label CLASS is discrimination for being poor or living in a marginal or poor place,label POLITICS is hatred of politicians, label DISABLED is discrimination against people with disabilities, label APPEARENCE is discrimination against people because of their physical appearance, specifically fatphobia, label CRIMINAL is discrimination against people because of a criminal record or links to criminality and label CALLS is hateful text that incites violence. If the text is not hatefull indicate label NOT HATEFUL"},
                {'role': 'user', 'content': row[text_column]}
                ]
        }
    }
    for _, row in df_test.iterrows()
]

In [21]:
batch[0]

{'custom_id': '397703',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'gpt-4o-mini',
  'messages': [{'role': 'system',
    'content': 'You must classify a text with one or more of the following labels: Label WOMEN discrimination against women, label LGBTI gender discrimination, specifically against gays, lesbians or transsexuals,label RACISM is racism, or discrimination against people according to their skin color,label CLASS is discrimination for being poor or living in a marginal or poor place,label POLITICS is hatred of politicians, label DISABLED is discrimination against people with disabilities, label APPEARENCE is discrimination against people because of their physical appearance, specifically fatphobia, label CRIMINAL is discrimination against people because of a criminal record or links to criminality and label CALLS is hateful text that incites violence. If the text is not hatefull indicate label NOT HATEFUL'},
   {'role': 'user', 'content': '@usuario 

In [22]:
# Save the batch to a .jsonl file
with open('batch_gpt_multiclass_v1.jsonl', 'w') as f:
    for entry in batch:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


In [23]:
batch_input_file = client.files.create(
  file=open("batch_gpt_multiclass_v1.jsonl", "rb"),
  purpose="batch"
)

In [24]:
batch_input_file

FileObject(id='file-p3GRq3qXolsAdETkRL4z96WH', bytes=12233663, created_at=1721929178, filename='batch_gpt_multiclass_v1.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [25]:
batch_input_file_id = 'file-p3GRq3qXolsAdETkRL4z96WH'

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

Batch(id='batch_RgCzgFJMbDxH1Hu0wEdKTWEq', completion_window='24h', created_at=1721929190, endpoint='/v1/chat/completions', input_file_id='file-p3GRq3qXolsAdETkRL4z96WH', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1722015590, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [26]:
client.batches.retrieve('batch_RgCzgFJMbDxH1Hu0wEdKTWEq')

Batch(id='batch_RgCzgFJMbDxH1Hu0wEdKTWEq', completion_window='24h', created_at=1721929190, endpoint='/v1/chat/completions', input_file_id='file-p3GRq3qXolsAdETkRL4z96WH', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-4o-mini in organization org-1uHjwiaB3OlPzoxfVzhqOSzs. Limit: 2,000,000 enqueued tokens. Please try again once some in_progress batches have been completed.', param=None)], object='list'), expired_at=None, expires_at=1722015590, failed_at=1721929194, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
batch_NQ9RgzBbLyqfmLOnoBYq0OPG